# Data required

- Churn [bool]
- Duration [int]
- Covariates [float]

# Import

In [3]:
def import_data_local_csv():
    import pandas as pd
    file_path = '/Users/patricksweeney/growth/04_Retention/04_Nonparametric churn prediction/Retention training.csv'
    data = pd.read_csv(file_path)
    return data

In [4]:
data = import_data_local_csv()
data.head()

,workspace_id,product,mrr_converted,mrr,expansion,annual,churned,start_date,end_date,t,...,tag_count_f,insight_count_f,reel_created_count_f,page_user_count_f,shared_object_note_count_f,shared_object_highlight_count_f,shared_object_tag_count_f,shared_object_insight_count_f,comment_count_f,reel_viewed_count_f
0,02deacf1-5198-483d-9bca-376ad30c22dd,Team,250.0,350.0,0.40,0,0,2022-02-11,2023-12-27,684.0,...,1.220760,0.264620,0.010234,1.494152,0.077485,0.0,0.00731,0.097953,0.0,0.067251
1,2f32ad80-80cf-454c-83b1-55d40e9c684b,Starter,18.0,18.0,0.00,0,1,2022-12-09,2023-03-08,89.0,...,0.146067,0.516854,0.000000,0.280899,0.000000,0.0,0.00000,0.000000,0.0,0.000000
2,2f671c70-030b-4202-a5fd-d6cb53b410a9,Starter,25.0,25.0,0.00,1,0,2023-11-03,2024-01-04,62.0,...,0.951613,0.048387,0.000000,0.838710,0.000000,0.0,0.00000,0.000000,0.0,0.016129
3,2fa16073-c492-45fe-8158-84ee254ff324,Team,200.0,270.0,0.35,1,1,2021-10-01,2023-07-03,548.0,...,0.235401,0.045620,0.000000,0.025547,0.000000,0.0,0.00000,0.001825,0.0,0.000000
4,38a47213-69f6-483d-8bf6-7d89d6951e7d,Starter,50.0,50.0,0.00,0,0,2023-05-08,2023-12-24,230.0,...,3.447826,0.056522,0.013043,0.613043,0.008696,0.0,0.00000,0.008696,0.0,0.000000


# Cox PH survival analysis

In [7]:
!pip install scikit-survival

DEPRECATION: Loading egg at /Users/patricksweeney/anaconda3/envs/EconML/lib/python3.11/site-packages/future-0.18.3-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /Users/patricksweeney/anaconda3/envs/EconML/lib/python3.11/site-packages/pyitlib-0.2.3-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.4/228.4 kB 6.4 MB/s eta 0:00:00
  Installing build dependencies ... error
  error: subprocess-exited-with-error
  
  × pip subprocess to install build dependencies did not run successfully.
  │ exit code: 1
  ╰─> [246 lines of output]


In [5]:
from sksurv.linear_model import CoxnetSurvivalAnalysis
from sksurv.util import Surv
import numpy as np

def cox_survival(data, duration_col, churn_col):
    # Extract the duration and churn columns
    duration = data[duration_col]
    churn = data[churn_col]

    # Create the structured array for the survival data
    structured_survival_data = Surv.from_arrays(event=churn, time=duration)

    # Extract covariates
    covariates = data.drop(columns=[duration_col, churn_col])

    # Initialize the Coxnet model with ridge penalty (alpha=1 for ridge)
    # l1_ratio = 0 corresponds to ridge regression
    model = CoxnetSurvivalAnalysis(l1_ratio=0, alpha_min_ratio=0.1)

    # Fit the model
    model.fit(covariates, structured_survival_data)

    # Predictions (returning for both training and a placeholder for test data)
    # For test predictions, replace 'covariates' with your test dataset
    train_predictions = model.predict(covariates)
    test_predictions = model.predict(covariates)  # Replace 'covariates' with test data

    return train_predictions, test_predictions

# Usage example:
train_predictions, test_predictions = cox_survival(data, 't', 'churn')


ModuleNotFoundError: No module named 'sksurv'

# AFT survival analysis